In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#import tensorflow as tf
import numpy as np
#import scipy.special as sci
import matplotlib.pyplot as plt
import pickle#5 as pickle
from scipy.linalg import expm, sinm, cosm,pinv
from numpy import linalg as LA
from sympy import *
from sklearn.decomposition import PCA
from scipy import integrate
import numpy.matlib as npm
import matplotlib as mpl
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH']='true'
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [3]:
## NN for phi
def neural_net(X, weights, biases):
    num_layers = len(weights) + 1  
    H=X
    for l in range(0,num_layers-2):
        W = weights[l]
        b = biases[l]
        H = tf.nn.tanh(tf.add(tf.matmul(H, W), b))  # Activation function tanh
    #    H = tf.nn.relu(tf.add(tf.matmul(H, W), b))  # Activation function ReLu
    W = weights[-1]
    b = biases[-1]
    Y =   tf.add(tf.matmul(H, W), b)
    return Y

# NN for g
def neural_net1(X, weights, biases):
    num_layers = len(weights) + 1  
    H=X
    for l in range(0,num_layers-2):
        W = weights[l]
        b = biases[l]
        H = tf.nn.tanh(tf.add(tf.matmul(H, W), b))  # Activation function tanh
    #    H = tf.nn.relu(tf.add(tf.matmul(H, W), b))  # Activation function ReLu
    W = weights[-1]
    b = biases[-1]
    Y =  tf.exp( tf.add(tf.matmul(H, W), b) )
    return Y

# Generate Gaussian random variable
def xavier_init(size):
    in_dim = size[0]
    out_dim = size[1]        
    xavier_stddev = np.sqrt(2.0/(in_dim + out_dim))
    return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev,dtype=tf.float32,seed=0), dtype=tf.float32)

In [4]:
#NN for g
def neural_net_g(x,y,t, weights, biases):
    u= neural_net1(tf.concat([x,y,t ],1), weights, biases )
    return u

#NN for phi
def neural_net_phi(t, weights_phi1, biases_phi1, weights_phi2, biases_phi2):
    phi1= neural_net(t, weights_phi1, biases_phi1)   #*t*(t1-t)+(phi_R-phi_L)/(t1-t0)*t+phi_L
    phi2= neural_net(t, weights_phi2, biases_phi2)   #*t*(t1-t)+(phi_R-phi_L)/(t1-t0)*t+phi_L
    return phi1,phi2

#### Analytical definition of SDE and object function

 $$ 
        b(x,y)=\left(\begin{array}{c}
            x-x^{3}- \beta x y^{2} \\-\left(1+x^{2}\right) y\end{array}
            \right).
$$

\begin{align*}
I(\varphi) \triangleq 
& \inf \left\{\int_{0}^{T} \int_{\mathbb{R}^{d}}(g(s, z) \ln g(s, z)-g(s, z)+1) \nu(d z) d s \mid g:[0, T] \longrightarrow[0, \infty)\right. \text { measurable: }\\  
&  \left.\varphi(t)=x+\int_{0}^{t} b(\varphi(s)) d s+\int_{0}^{t} \int_{\mathbb{R}^{d}} z(g(s, z)-1) \nu(d z) d s, \quad t \in[0, T]\right\}
\end{align*}

$t=0:\delta t:T$ and $t_i=i*\delta t$, where $i=0,1,...N$ and $T=N\delta t$

$z_1=-L:\delta z_1:L$ and $z_{1j}=-L+j\delta z_1$, where $j=0,1,...M$ and $2L=M\delta z_1$

$z_2=-L:\delta z_2:L$ and $z_{2k}=-L+j\delta z_2$, where $k=0,1,...M$ and $2L=M\delta z_2$
$$
\begin{aligned}
& \int_{0}^{T} \int_{\mathbb{R}^{2}}\big[g(s, z_1,z_2) \ln g(s, z_1,z_2)-g(s, z_1,z_2)+1\big] \exp\big( -(z_1^2+z_2^2)^{\frac{\alpha}{2}} 
\big) dz_1 dz_2 d s\\
=&\delta t \delta z_1 \delta z_2 \sum_{i=0}^{N} \sum_{j,k=0}^{M} (g(s_i, z_{1j}, z_{2k}) \ln g(s_i, z_{1j}, z_{2k})-g(s_i, z_{1j}, z_{2k})+1) e^{-(z_{1j}^2+ z_{2k}^2)^{\frac{\alpha}{2} }} 
\end{aligned}
$$
$$
\begin{aligned}
        \varphi_1(t) & = x_1^{(1)}+ \int_0^t \big(\varphi_1(s)-\varphi_1^3(s)-\varphi_1(s)\varphi_2^2(s) \big)ds \\
        & + \int_0^t \int_{\mathbb{R}^2 \backslash \{ 0\}}z_1\big( g(s,z_1,z_2)-1 \big)\exp\big( -(z_1^2+z_2^2)^{\frac{3}{4}} \big) dz_1 dz_2 ds, \\
        \varphi_2(t) & = x_1^{(2)}- \int_0^t \big( 1+\varphi_1^2(s) \big) \varphi_2(s) ds \\
        & + \int_0^t \int_{\mathbb{R}^2 \backslash \{ 0\}}z_2 \big( g(s,z_1,z_2)-1 \big) \exp\big( -(z_1^2+z_2^2)^{\frac{3}{4}} \big) dz_1 dz_2 ds.        
\end{aligned}
$$

In [5]:
def potential(x,y):
    gamma=10
    #U= -1/2*x**2+1/4*x**4+1/2*y**2+1/2*x**2*y**2
    U_x=x**3-x+gamma*x*y**2
    U_y=y+x**2*y
    return U_x,U_y

In [6]:
#Given z=(x,y),t, Output integrand z(g(t,z)-1)exp(-|z|^\alpha)
def net_ode_int(x,y,t, weights_g, biases_g, alpha ):
    g= neural_net_g(x,y,t, weights_g, biases_g)     
    f1=x*(g-1) *tf.exp(-( x**2+y**2  )**(alpha/2 ) )
    f2=y*(g-1) *tf.exp(-( x**2+y**2  )**(alpha/2 )  )
    return f1,f2
# Generate \varphi_t-b(\varphi)
def net_ode_phi(t, weights_phi1, biases_phi1, weights_phi2, biases_phi2 ):
    phi1=neural_net_phi(t, weights_phi1, biases_phi1, weights_phi2, biases_phi2)[0]
    phi2=neural_net_phi(t, weights_phi1, biases_phi1, weights_phi2, biases_phi2)[1]
    phi1_t = tf.gradients(phi1, t)[0]
    phi2_t = tf.gradients(phi2, t)[0]
    U_x=potential( phi1, phi2)
    return phi1_t+U_x[0],phi2_t+U_x[1]

#Given z=(x,y),t, Output object integrand (g(t,z)log(g(t,z))-g(t,z)+1)exp(-|z|^\alpha)
def net_opt_g(x,y,t,weights_g,biases_g,alpha  ):
    g= neural_net_g(x,y,t, weights_g, biases_g)
    f= ( g*tf.log(g)-g+1 )*tf.exp(-( x**2+y**2 )**(alpha/2) )
    return f

In [7]:
layers_g = [3] + 2* [20] + [1]
L_g = len(layers_g)
np.random.seed(0)
weights_g = [xavier_init([layers_g[l], layers_g[l+1]]) for l in range(0, L_g-1)]    
biases_g = [tf.Variable( tf.zeros((1, layers_g[l+1]),dtype=tf.float32)) for l in range(0, L_g-1)]

layers_phi = [1] + 2* [20] + [1]
L_phi = len(layers_phi)
weights_phi1 = [xavier_init([layers_phi[l], layers_phi[l+1]]) for l in range(0, L_phi-1)]    
biases_phi1 = [tf.Variable( tf.zeros((1, layers_phi[l+1]),dtype=tf.float32)) for l in range(0, L_phi-1)]

weights_phi2 = [xavier_init([layers_phi[l], layers_phi[l+1]]) for l in range(0, L_phi-1)]    
biases_phi2= [tf.Variable( tf.zeros((1, layers_phi[l+1]),dtype=tf.float32)) for l in range(0, L_phi-1)]

reg1=0
for i in range(L_phi-1):
    reg1=reg1+tf.nn.l2_loss(weights_phi1[i])
 

reg2=0
for i in range(L_phi-1):
    reg2=reg2+tf.nn.l2_loss(weights_phi2[i])
 

In [8]:
# Generate time mesh
t0=0
t1=100
Nt=2000
dt=(t1-t0)/Nt
vect=np.linspace(t0,t1,Nt+1)[:,None]
vect_tf = tf.cast(vect,dtype=tf.float32)

In [9]:
# Generate integration mesh
alpha=1.5
L=5
x0=-L
x1=L
y0=-L
y1=L
Nx=100
dx=(x1-x0)/Nx
Ny=100
dy=(y1-y0)/Ny
vecx=np.linspace(x0,x1,Nx+1)[:,None]
vecy=np.linspace(y0,y1,Ny+1)[:,None]
vecx_tf = tf.cast(vecx,dtype=tf.float32)
vecy_tf = tf.cast(vecy,dtype=tf.float32)

In [10]:
# Generate a tensor to store time-space mesh xff={(t_i,x_j,y_k)}
counter=0
xff = np.zeros((len(vect)*len(vecx) *len(vecy) ,3),dtype=np.float32)
for i in range(len(vect)):
    for j in range(len(vecx)):
        for k in range(len(vecy)):
            xff[counter,:]=[vect[i],vecx[j],vecy[k] ]
            counter=counter+1
tf1=xff[:,0:1]
xf=xff[:,1:2] 
yf=xff[:,2:3] 

In [12]:
tf1_tf = tf.cast(tf1 , dtype=tf.float32)
xf_tf = tf.cast(xf , dtype=tf.float32)
yf_tf = tf.cast(yf , dtype=tf.float32)

In [13]:
#(phi1_L,phi2_L) to (phi1_R,phi2_R)
phi1_L=-1  #t0
phi2_L=0

phi1_R=1   #t1
phi2_R=0


phi1L_nn=neural_net_phi(vect_tf,weights_phi1,biases_phi1,weights_phi2,biases_phi2)[0][0] 
phi2L_nn=neural_net_phi(vect_tf,weights_phi1,biases_phi1,weights_phi2,biases_phi2)[1][0] 

phi1R_nn=neural_net_phi(vect_tf,weights_phi1,biases_phi1,weights_phi2,biases_phi2)[0][-1] 
phi2R_nn=neural_net_phi(vect_tf,weights_phi1,biases_phi1,weights_phi2,biases_phi2)[1][-1] 
# loss boundary
loss_b=tf.reduce_mean(tf.square(phi1L_nn-phi1_L))+ tf.reduce_mean(tf.square(phi1R_nn-phi1_R  ))+tf.reduce_mean(tf.square(phi2L_nn-phi2_L))+ tf.reduce_mean(tf.square(phi2R_nn-phi2_R  ))#+       tf.reduce_mean(tf.square(phi1M_nn-phi1_M))+ tf.reduce_mean(tf.square(phi2M_nn-phi2_M  ))

In [15]:
nu_ode_int=net_ode_int(xf_tf,yf_tf,tf1_tf, weights_g, biases_g,alpha)

In [16]:
loss_phi1=dt*tf.reduce_sum(tf.square( net_ode_phi(vect_tf,weights_phi1,biases_phi1,weights_phi2,biases_phi2)[0]-dx*dy*tf.reduce_sum(tf.reshape(nu_ode_int[0], shape=(Nt+1,(Nx+1)*(Ny+1) )  ) ,1)[:,None] ))

loss_phi2=dt*tf.reduce_sum(tf.square( net_ode_phi(vect_tf,weights_phi1,biases_phi1,weights_phi2,biases_phi2 )[1]-dx*dy*tf.reduce_sum(tf.reshape(nu_ode_int[1], shape=(Nt+1,(Nx+1)*(Ny+1) )  ) ,1)[:,None] ))

loss_phi=loss_phi1+loss_phi2+100*loss_b

loss_g=dx*dy*dt*tf.reduce_sum(  net_opt_g(xf_tf,yf_tf,tf1_tf,weights_g,biases_g ,alpha ) )

loss=1*loss_g+0.1*loss_phi#+0.001*(reg1+reg2)

In [17]:
optimizer_Adam1 = tf.train.AdamOptimizer(1e-3)
train_op_Adam1 = optimizer_Adam1.minimize(loss_phi)

optimizer_Adam2 = tf.train.AdamOptimizer(1e-3)
train_op_Adam2 = optimizer_Adam2.minimize(loss_g)

optimizer_Adam3 = tf.train.AdamOptimizer(1e-3)
train_op_Adam3 = optimizer_Adam3.minimize(loss)

In [18]:
loss_g_record = []
loss_phi_record = []
loss_phi1_record = []
loss_phi2_record = []
loss_b_record=[]
saver = tf.train.Saver(max_to_keep=1000)
savedir='xiaoli'
config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
min_loss = 1e16

#### Training 

In [19]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    g_pred=neural_net_g(xf_tf,yf_tf,tf1_tf, weights_g, biases_g)
    phi_pred=neural_net_phi(vect_tf,weights_phi1,biases_phi1,weights_phi2,biases_phi2) 
    for i in range(200001):
        sess.run(train_op_Adam3 )
#         sess.run(train_op_Adam1 )
#         if i%10==0:
#             sess.run(train_op_Adam2 )
        if i % 100 == 0:
            (loss_result,loss_g_result, loss_phi_result ,  loss_phi1_result , loss_phi2_result ,loss_b_result ) = sess.run([loss,loss_g, loss_phi,loss_phi1,loss_phi2,loss_b])
            (phi1L_np,phi1R_np,phi2L_np,phi2R_np)=sess.run([phi1L_nn,phi1R_nn,phi2L_nn,phi2R_nn])
            g_t0= sess.run(g_pred)
            phi_t0= sess.run(phi_pred)
            loss_g_record.append(loss_g_result)
            loss_phi_record.append(loss_phi_result)
            loss_phi1_record.append(loss_phi1_result)
            loss_phi2_record.append(loss_phi2_result)
            loss_b_record.append(loss_b_result)
            if loss_result<min_loss:
                min_loss=loss_result
                g_opt= sess.run(g_pred) 
                phi_opt= sess.run(phi_pred)
                i_opt=i
           # temp_loss=sess.run(loss, feed_dict = all_dict)
            print ('  %d  %8.2e  %8.2e  %8.2e  %8.2e  %8.2e  %8.2e  %8.2e  %8.2e   %8.2e  %8.2e ' % (i, loss_result,loss_g_result,loss_phi_result,loss_phi1_result,loss_phi2_result,loss_b_result,phi1L_np,phi1R_np,phi2L_np,phi2R_np) )
            #save_path = saver.save(sess, savedir+'/' + str(i) + '.ckpt')
       
        if i % 10000 ==0:
            (weights_g_np,biases_g_np,weights_phi1_np,biases_phi1_np,weights_phi2_np,biases_phi2_np )=sess.run([weights_g,biases_g,weights_phi1,biases_phi1, weights_phi2,biases_phi2 ])
            sample_list = {"weights_g": weights_g_np, "biases_g": biases_g_np,"weights_phi1": weights_phi1_np, "biases_phi1": biases_phi1_np,"weights_phi2": weights_phi2_np, "biases_phi2": biases_phi2_np}
            file_name = './result/hyper'+str(i)+'.pkl'
            open_file = open(file_name, "wb")
            pickle.dump(sample_list, open_file)
            open_file.close()

            np.savetxt('./result/loss_g-mat.txt',np.array(loss_g_record),fmt='%10.5e')
            np.savetxt('./result/loss_phi-mat.txt',np.array(loss_phi_record),fmt='%10.5e')
            np.savetxt('./result/loss_b-mat.txt',np.array(loss_b_record),fmt='%10.5e')


  0  7.01e+02  2.27e+02  4.74e+03  3.98e+03  2.17e+02  5.34e+00  6.63e-03  -8.83e-01   1.12e-02  -8.83e-01 
  100  7.90e+00  5.52e-01  7.35e+01  6.12e+00  2.00e-01  6.72e-01  -3.96e-01  4.46e-01   3.43e-04  3.21e-02 
  200  1.75e+00  3.26e-01  1.42e+01  4.52e+00  4.05e-01  9.33e-02  -7.33e-01  8.57e-01   4.29e-04  3.61e-02 
  300  3.69e-01  1.29e-01  2.40e+00  1.18e+00  2.41e-01  9.80e-03  -9.09e-01  1.04e+00   2.08e-04  8.12e-03 
  400  2.42e-01  9.79e-02  1.44e+00  1.00e+00  2.45e-01  1.93e-03  -9.74e-01  1.03e+00   4.46e-04  5.11e-03 
  500  1.88e-01  7.61e-02  1.12e+00  7.85e-01  2.40e-01  9.78e-04  -9.92e-01  1.03e+00   5.95e-04  4.61e-03 
  600  1.50e-01  6.07e-02  8.89e-01  6.23e-01  2.19e-01  4.68e-04  -9.96e-01  1.02e+00   8.08e-04  4.40e-03 
  700  1.28e-01  5.12e-02  7.71e-01  5.60e-01  1.95e-01  1.68e-04  -9.97e-01  1.01e+00   1.11e-03  5.03e-03 
  800  1.19e-01  4.62e-02  7.27e-01  5.42e-01  1.76e-01  7.93e-05  -9.98e-01  1.01e+00   1.52e-03  6.00e-03 
  900  1.14e-01  4.3

  8400  5.50e-02  2.02e-02  3.49e-01  1.85e-01  1.63e-01  3.56e-06  -1.00e+00  1.00e+00   1.40e-03  -1.21e-03 
  8500  5.50e-02  2.02e-02  3.48e-01  1.85e-01  1.63e-01  3.65e-06  -1.00e+00  1.00e+00   1.51e-03  -1.09e-03 
  8600  5.48e-02  2.01e-02  3.47e-01  1.83e-01  1.63e-01  3.72e-06  -1.00e+00  1.00e+00   1.47e-03  -1.15e-03 
  8700  5.48e-02  2.03e-02  3.45e-01  1.81e-01  1.64e-01  3.21e-06  -1.00e+00  1.00e+00   1.38e-03  -1.03e-03 
  8800  5.47e-02  2.01e-02  3.45e-01  1.81e-01  1.64e-01  4.36e-06  -9.99e-01  1.00e+00   1.66e-03  -1.15e-03 
  8900  5.47e-02  2.04e-02  3.43e-01  1.80e-01  1.63e-01  3.43e-06  -1.00e+00  1.00e+00   1.47e-03  -1.02e-03 
  9000  5.45e-02  2.01e-02  3.44e-01  1.80e-01  1.64e-01  2.47e-06  -9.99e-01  1.00e+00   9.37e-04  -1.13e-03 
  9100  5.45e-02  2.00e-02  3.45e-01  1.79e-01  1.65e-01  2.35e-06  -1.00e+00  1.00e+00   1.24e-03  -7.73e-04 
  9200  5.43e-02  2.01e-02  3.42e-01  1.77e-01  1.65e-01  3.74e-06  -1.00e+00  1.00e+00   1.64e-03  -9.17e-04 
 

  15800  4.27e-02  1.64e-02  2.64e-01  9.28e-02  1.71e-01  1.36e-06  -9.99e-01  1.00e+00   5.35e-04  6.96e-04 
  15900  4.30e-02  1.66e-02  2.63e-01  9.33e-02  1.70e-01  2.20e-06  -9.99e-01  9.99e-01   6.93e-04  -2.93e-05 
  16000  4.33e-02  1.63e-02  2.69e-01  9.07e-02  1.78e-01  1.07e-05  -9.99e-01  9.99e-01   1.89e-04  2.95e-03 
  16100  4.30e-02  1.68e-02  2.61e-01  9.29e-02  1.68e-01  9.65e-07  -1.00e+00  1.00e+00   5.64e-04  -4.59e-04 
  16200  4.26e-02  1.63e-02  2.62e-01  9.34e-02  1.69e-01  9.09e-07  -1.00e+00  1.00e+00   5.83e-04  -6.81e-04 
  16300  4.27e-02  1.64e-02  2.64e-01  9.39e-02  1.69e-01  4.75e-06  -1.00e+00  1.00e+00   9.80e-04  -1.93e-03 
  16400  4.25e-02  1.63e-02  2.62e-01  9.22e-02  1.70e-01  9.07e-07  -1.00e+00  9.99e-01   6.95e-04  -9.29e-05 
  16500  4.26e-02  1.64e-02  2.62e-01  9.47e-02  1.67e-01  1.66e-06  -1.00e+00  1.00e+00   2.29e-04  -5.32e-04 
  16600  4.26e-02  1.63e-02  2.63e-01  9.40e-02  1.68e-01  3.14e-06  -9.99e-01  9.99e-01   7.26e-04  -1.17

  23200  4.21e-02  1.62e-02  2.59e-01  9.03e-02  1.69e-01  2.15e-06  -1.00e+00  9.99e-01   -6.71e-06  6.12e-04 
  23300  4.21e-02  1.62e-02  2.59e-01  8.88e-02  1.70e-01  2.39e-06  -1.00e+00  1.00e+00   4.85e-05  1.25e-03 
  23400  4.21e-02  1.62e-02  2.59e-01  8.79e-02  1.71e-01  2.07e-06  -1.00e+00  1.00e+00   2.90e-04  1.13e-03 
  23500  4.20e-02  1.62e-02  2.58e-01  9.04e-02  1.68e-01  1.64e-07  -1.00e+00  1.00e+00   3.02e-04  -2.43e-04 
  23600  4.20e-02  1.62e-02  2.58e-01  8.95e-02  1.69e-01  1.71e-07  -1.00e+00  1.00e+00   3.02e-04  -2.71e-05 
  23700  4.35e-02  1.61e-02  2.74e-01  8.88e-02  1.82e-01  3.32e-05  -9.99e-01  9.96e-01   2.60e-04  3.94e-03 
  23800  4.20e-02  1.62e-02  2.58e-01  9.13e-02  1.67e-01  5.94e-07  -1.00e+00  1.00e+00   -1.84e-05  -1.37e-04 
  23900  4.20e-02  1.61e-02  2.59e-01  8.87e-02  1.70e-01  4.43e-07  -1.00e+00  1.00e+00   5.09e-04  7.36e-05 
  24000  4.23e-02  1.62e-02  2.61e-01  9.17e-02  1.69e-01  6.15e-06  -9.99e-01  9.98e-01   2.72e-04  -3.09e

  30600  4.19e-02  1.61e-02  2.58e-01  8.81e-02  1.69e-01  5.54e-07  -1.00e+00  1.00e+00   7.26e-04  -1.40e-05 
  30700  4.19e-02  1.62e-02  2.57e-01  8.83e-02  1.69e-01  2.35e-07  -1.00e+00  1.00e+00   3.63e-04  -1.60e-04 
  30800  4.19e-02  1.62e-02  2.58e-01  8.86e-02  1.69e-01  3.22e-07  -1.00e+00  1.00e+00   3.15e-04  8.19e-05 
  30900  4.25e-02  1.66e-02  2.58e-01  8.95e-02  1.69e-01  5.96e-07  -1.00e+00  9.99e-01   1.12e-04  4.55e-05 
  31000  4.19e-02  1.62e-02  2.57e-01  8.87e-02  1.69e-01  3.96e-07  -1.00e+00  1.00e+00   3.40e-04  -9.51e-05 
  31100  4.21e-02  1.62e-02  2.60e-01  8.98e-02  1.69e-01  5.74e-06  -1.00e+00  9.98e-01   3.96e-04  -1.81e-04 
  31200  4.21e-02  1.63e-02  2.58e-01  8.95e-02  1.69e-01  1.64e-07  -1.00e+00  1.00e+00   3.05e-04  -2.49e-04 
  31300  4.20e-02  1.62e-02  2.58e-01  8.91e-02  1.69e-01  1.29e-06  -1.00e+00  1.00e+00   1.82e-04  1.21e-04 
  31400  4.31e-02  1.75e-02  2.56e-01  8.89e-02  1.67e-01  1.41e-07  -1.00e+00  1.00e+00   -5.35e-05  3.72e

...

  170200  4.17e-02  1.61e-02  2.56e-01  9.00e-02  1.66e-01  1.68e-06  -9.99e-01  9.99e-01   -2.01e-05  -5.92e-04 
  170300  4.17e-02  1.61e-02  2.56e-01  8.80e-02  1.68e-01  3.31e-07  -1.00e+00  1.00e+00   3.18e-04  -4.15e-04 
  170400  4.17e-02  1.62e-02  2.56e-01  9.13e-02  1.64e-01  6.65e-07  -1.00e+00  1.00e+00   -5.63e-04  -1.12e-04 
  170500  4.20e-02  1.65e-02  2.55e-01  8.54e-02  1.70e-01  9.31e-09  -1.00e+00  1.00e+00   8.63e-06  5.43e-05 
  170600  4.17e-02  1.61e-02  2.56e-01  8.56e-02  1.71e-01  4.52e-07  -1.00e+00  1.00e+00   -2.29e-04  6.30e-04 
  170700  4.17e-02  1.62e-02  2.55e-01  8.85e-02  1.67e-01  3.81e-08  -1.00e+00  1.00e+00   1.68e-05  -9.31e-05 
  170800  4.17e-02  1.61e-02  2.56e-01  8.81e-02  1.68e-01  3.06e-08  -1.00e+00  1.00e+00   8.51e-05  -1.12e-04 
  170900  4.17e-02  1.61e-02  2.56e-01  8.60e-02  1.70e-01  4.05e-07  -1.00e+00  1.00e+00   -3.95e-04  4.87e-04 
  171000  4.17e-02  1.62e-02  2.56e-01  8.77e-02  1.68e-01  1.63e-07  -1.00e+00  1.00e+00   1.0

  177500  4.17e-02  1.62e-02  2.56e-01  8.53e-02  1.70e-01  1.16e-06  -1.00e+00  1.00e+00   2.59e-04  -6.09e-05 
  177600  4.17e-02  1.61e-02  2.56e-01  8.51e-02  1.71e-01  8.23e-07  -1.00e+00  1.00e+00   8.56e-04  -2.81e-04 
  177700  4.17e-02  1.61e-02  2.56e-01  8.76e-02  1.68e-01  9.81e-08  -1.00e+00  1.00e+00   -2.25e-04  2.17e-04 
  177800  4.17e-02  1.61e-02  2.56e-01  8.69e-02  1.69e-01  2.97e-08  -1.00e+00  1.00e+00   1.55e-04  -2.18e-06 
  177900  4.17e-02  1.62e-02  2.56e-01  8.96e-02  1.66e-01  1.25e-07  -1.00e+00  1.00e+00   4.44e-05  -2.34e-04 
  178000  4.17e-02  1.62e-02  2.55e-01  9.12e-02  1.64e-01  2.34e-07  -1.00e+00  1.00e+00   -3.26e-04  -3.46e-04 
  178100  4.17e-02  1.62e-02  2.55e-01  8.77e-02  1.67e-01  1.27e-08  -1.00e+00  1.00e+00   -1.64e-05  -3.07e-05 
  178200  4.17e-02  1.61e-02  2.56e-01  8.69e-02  1.69e-01  2.03e-07  -1.00e+00  1.00e+00   -9.68e-05  -5.35e-06 
  178300  4.17e-02  1.61e-02  2.56e-01  8.44e-02  1.71e-01  3.57e-07  -1.00e+00  1.00e+00   4

  184800  4.17e-02  1.61e-02  2.56e-01  8.80e-02  1.68e-01  1.33e-08  -1.00e+00  1.00e+00   8.63e-05  -3.19e-05 
  184900  4.17e-02  1.61e-02  2.56e-01  8.83e-02  1.67e-01  9.59e-08  -1.00e+00  1.00e+00   -3.26e-05  2.09e-05 
  185000  4.17e-02  1.61e-02  2.56e-01  8.98e-02  1.66e-01  4.66e-07  -9.99e-01  1.00e+00   -6.04e-05  -9.23e-05 
  185100  4.17e-02  1.61e-02  2.56e-01  8.76e-02  1.68e-01  2.61e-07  -1.00e+00  1.00e+00   2.14e-04  -1.59e-04 
  185200  4.17e-02  1.61e-02  2.56e-01  8.79e-02  1.68e-01  1.67e-07  -1.00e+00  1.00e+00   2.12e-04  -1.30e-04 
  185300  4.19e-02  1.62e-02  2.57e-01  8.70e-02  1.69e-01  5.59e-06  -9.98e-01  9.98e-01   -2.88e-04  5.33e-04 
  185400  4.17e-02  1.61e-02  2.56e-01  8.49e-02  1.71e-01  2.72e-07  -1.00e+00  1.00e+00   2.28e-04  4.69e-04 
  185500  4.17e-02  1.61e-02  2.56e-01  8.56e-02  1.70e-01  1.47e-06  -1.00e+00  1.00e+00   8.85e-04  -5.16e-04 
  185600  4.18e-02  1.62e-02  2.55e-01  8.57e-02  1.70e-01  3.95e-07  -1.00e+00  1.00e+00   -3.7

  192100  4.17e-02  1.62e-02  2.56e-01  9.05e-02  1.65e-01  3.09e-07  -1.00e+00  1.00e+00   -5.27e-04  1.37e-04 
  192200  4.17e-02  1.61e-02  2.56e-01  9.02e-02  1.66e-01  2.44e-07  -1.00e+00  1.00e+00   -3.53e-04  -3.34e-05 
  192300  4.17e-02  1.61e-02  2.56e-01  8.72e-02  1.69e-01  4.52e-07  -1.00e+00  1.00e+00   4.66e-04  -3.97e-04 
  192400  4.17e-02  1.61e-02  2.56e-01  8.99e-02  1.66e-01  1.52e-07  -1.00e+00  1.00e+00   -1.29e-04  -5.69e-05 
  192500  4.20e-02  1.63e-02  2.57e-01  8.92e-02  1.67e-01  4.26e-07  -9.99e-01  1.00e+00   -9.09e-05  -1.84e-04 
  192600  4.17e-02  1.62e-02  2.56e-01  9.13e-02  1.65e-01  9.27e-07  -1.00e+00  1.00e+00   3.33e-04  -8.48e-04 
  192700  4.17e-02  1.62e-02  2.55e-01  8.89e-02  1.66e-01  8.30e-08  -1.00e+00  1.00e+00   -5.79e-05  -1.56e-04 
  192800  4.17e-02  1.62e-02  2.56e-01  8.84e-02  1.67e-01  5.52e-07  -1.00e+00  1.00e+00   -2.88e-04  1.71e-04 
  192900  4.17e-02  1.61e-02  2.56e-01  8.59e-02  1.70e-01  1.36e-07  -1.00e+00  1.00e+00   

  199400  4.20e-02  1.66e-02  2.55e-01  8.74e-02  1.67e-01  3.83e-08  -1.00e+00  1.00e+00   -4.53e-05  6.04e-06 
  199500  4.18e-02  1.61e-02  2.56e-01  8.50e-02  1.71e-01  6.56e-07  -1.00e+00  1.00e+00   -2.73e-04  7.32e-04 
  199600  4.20e-02  1.64e-02  2.56e-01  8.71e-02  1.68e-01  2.12e-08  -1.00e+00  1.00e+00   1.44e-04  -5.98e-06 
  199700  4.17e-02  1.62e-02  2.56e-01  8.75e-02  1.68e-01  3.72e-08  -1.00e+00  1.00e+00   1.70e-04  -1.20e-05 
  199800  4.19e-02  1.64e-02  2.55e-01  8.70e-02  1.68e-01  2.26e-07  -1.00e+00  1.00e+00   2.11e-04  -1.34e-04 
  199900  4.17e-02  1.61e-02  2.55e-01  8.62e-02  1.69e-01  7.58e-08  -1.00e+00  1.00e+00   -5.89e-05  1.66e-04 
  200000  4.17e-02  1.61e-02  2.56e-01  8.64e-02  1.70e-01  2.30e-07  -1.00e+00  1.00e+00   1.34e-04  1.22e-04 


In [ ]:
vecx = np.linspace(-1,1,201)[:,None]
risline = np.ones((len(vecx) ,1),dtype=np.float32)
rislineup = risline*(0.3)
rislinedown = risline*(0)
fig=  plt.figure()
plt.plot(phi_opt[0],phi_opt[1],'r-', label='$\phi_{NN}$' ) 
plt.plot(vecx,rislineup,'g--' )
plt.plot(vecx,rislinedown,'g--' )
plt.xlabel('$\phi_1$')
plt.ylabel('$\phi_2$')
plt.legend()
plt.show()
fig.savefig('MTP_Levy_alpha=15T'+ str(t1) + '.png', dpi=600,format='png')

In [ ]:
vecx = np.linspace(-1,1,201)[:,None]
risline = np.ones((len(vecx) ,1),dtype=np.float32)
rislineup = risline*(0.3)
rislinedown = risline*(0)
fig=  plt.figure()
plt.plot(phi[0],phi[1],'r-', label='$\phi_{NN}$' ) 
plt.plot(vecx,rislineup,'g--' )
plt.plot(vecx,rislinedown,'g--' )
plt.xlabel('$\phi_1$')
plt.ylabel('$\phi_2$')
plt.legend()
plt.show()
fig.savefig('MTP_Levy_alpha=15T'+ str(t1) + '.png', dpi=600,format='png')

In [ ]:
mkdir result